# Harmonizing data located within and outside of the NASA Earthdata Cloud

---

## Timing

- Exercise: 45 min


---

## Summary

This tutorial will combine several workflow steps and components from the previous days, demonstrating the process of using the geolocation of data available outside of the Earthdata Cloud to then access coincident variables of cloud-accessible data. This may be a common use case as NASA Earthdata continues to migrate to the cloud, producing a "hybrid" data archive across Amazon Web Services (AWS) and original on-premise data storage systems. Additionally, you may also want to combine field measurements with remote sensing data available on the Earthdata Cloud.

This specific example explores the harmonization of the ICESat-2 ATL03 data product, currently (as of November 2021) available publicly via direct download at the NSIDC DAAC, with Sea Surface Temperature variables available from PO.DAAC on the Earthdata Cloud. 


### Objectives

[TODO]

---

### Import packages

In [ ]:
import requests
import netrc
from pprint import pprint
import os
from pathlib import Path

import s3fs

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# packages below here are not used
#from xml.etree import ElementTree as ET
#import time
#import zipfile
#import io
#import shutil
#import json
#from urllib import request

### Determine storage location of datasets of interest

First, let's see whether our datasets of interest reside in the Earthdata Cloud or whether they reside on premise, or "on prem" at a local data center.

Background from CMR API [TODO: consider removing]:
The cloud_hosted parameter can be set to “true” or “false”. When true, the results will be restricted to collections that have a DirectDistributionInformation element or have been tagged with gov.nasa.earthdatacloud.s3.

We are building off of the CMR introductory tutorial, beginning with a collection search.

In [ ]:
cmr_search_url = 'https://cmr.earthdata.nasa.gov/search'

We want to search by collection to inspect the access and service options that exist:

In [ ]:
cmr_collection_url = f'{cmr_search_url}/{"collections"}'

In the CMR introduction tutorial, we explored cloud-hosted collections from different DAAC providers, and identified the CMR concept-id for a given dataset id (also referred to as a short_name). Here we'll start with two datasets that we want to explore over a coincident area and time:

In [ ]:
modis_name = 'MODIS_A-JPL-L2P-v2019.0'
icesat2_name = 'ATL03'

Like in the intro tutorial, we're going to first determine what concept-ids are returned for the MODIS dataset. First, retrieve collection results based on the MODIS `short_name`:

In [ ]:
response = requests.get(cmr_collection_url, 
                        params={
                            'short_name': modis_name,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

For each collection result, print out the CMR concept-id and version:

In [ ]:
collections = response['feed']['entry']

for collection in collections:
    print(f'{collection["id"]} {"version:"}{collection["version_id"]}')

Two collections are returned, both at version 2019.0. We can see from the suffix of the id that one is associated with "POCLOUD" versus "PODAAC". That gives us a clue in terms of where the data are hosted, but we can also use the `cloud_hosted` parameter set to True to confirm.

In [ ]:
response = requests.get(cmr_collection_url, 
                        params={
                            'short_name': modis_name,
                            'cloud_hosted': 'True',
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

In [ ]:
collections = response['feed']['entry']

for collection in collections:
    print(f'{collection["id"]} {"version:"}{collection["version_id"]}')

We will save this concept-id to use later on when we access the data granules.

In [ ]:
modis_cloud_id = collections[0]["id"]

Now we will try our ICESat-2 dataset to see what id's are returned for a given dataset name.

In [ ]:
response = requests.get(cmr_collection_url, 
                        params={
                            'short_name': icesat2_name,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

In [ ]:
collections = response['feed']['entry']

for collection in collections:
    print(f'{collection["id"]} {"version:"}{collection["version_id"]}')

Two separate datasets exist in the CMR, one at version 3 and one at version 4. Let's see if these are `cloud_hosted`:

In [ ]:
response = requests.get(cmr_collection_url, 
                        params={
                            'short_name': icesat2_name,
                            'cloud_hosted': 'False',
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

In [ ]:
collections = response['feed']['entry']

for collection in collections:
    print(f'{collection["id"]} {"version:"}{collection["version_id"]}')

When set to `False`, we get our collections back. We have now determined that we have a copy of the MODIS dataset in the cloud, whereas the ICESat-2 dataset (both versions) remains "on premise", residing in a local data center. 

Save the ATL03 concept ID and the MODIS GHRSST concept ID to variables:

In [ ]:
icesat2_concept_id = 'C1997321091-NSIDC_ECS'
modis_concept_id = 'C1940475563-POCLOUD'

#### Specify time range and area of interest 

We are going to focus on getting data for an area north of Greenland for a single day in June.

These `bounding_box` and `temporal` variables will be used for data search, subset, and access below

In [ ]:
# Bounding Box spatial parameter in decimal degree 'W,S,E,N' format.
bounding_box = '-62.8,81.7,-56.4,83'

# Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format
temporal = '2019-06-22T00:00:00Z,2019-06-22T23:59:59Z'

Perform a granule search over our time and area of interest. How many granules are returned?

In [ ]:
granule_url = f'{cmr_search_url}/{"granules"}'

In [ ]:
response = requests.get(granule_url, 
                        params={
                            'concept_id': icesat2_concept_id,
                            'temporal': temporal,
                            'bounding_box': bounding_box,
                            'page_size': 200,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response.headers['CMR-Hits'])

Print the file names, size, and links:

In [ ]:
granules = response.json()['feed']['entry']

for granule in granules:
    print(f'{granule["producer_granule_id"]} {granule["granule_size"]} {granule["links"][0]["href"]}')

### Download ICESat-2 ATL03 granule
[TODO] Describe what services are available, including icepyx (provide references), but just direct download for simplicity. Describe that this is being "downloaded" to our cloud environment - what does that mean in terms of cost, etc.

We've found 2 granules.  We'll download the first one and write it to a file with the same name as the `producer_granule_id`.

We need the url for the granule as well.  This is `href` links we printed out above.

In [ ]:
icesat_id = granules[0]["producer_granule_id"]
icesat_url = granules[0]['links'][0]['href']

You need Earthdata login credentials to download data from NASA DAACs.  These are the credentials you stored in the `.netrc` file you setup in previous tutorials.  

We'll use the `netrc` package to retrieve your login and password without exposing them.

In [ ]:
info = netrc.netrc()
login, account, password = info.authenticators('urs.earthdata.nasa.gov')

To retrieve the granule data, we use the `requests.get()` method, passing Earthdata login credentials as a `tuple` using the `auth` keyword.

In [ ]:
r = requests.get(icesat_url, auth=(login, password))

The response returned by requests has the same structure as all the other responses: a header and contents.  The header information has information about the response, including the size of the data we downloaded in bytes. 

In [ ]:
for k, v in r.headers.items():
    print(f'{k}: {v}')

The contents needs to be saved to a file.  To keep the directory clean, we will create a `downloads` directory to store the file.  We can use a shell command to do this or use the `mkdir` method from the `os` package. 

In [ ]:
os.mkdir('downloads')

You should see a `downloads` directory in the file browser.

To write the data to a file, we use `open` to open a file.  We need to specify that the file is open for writing by using the _write-mode_ `w`.  We also need to specify that we want to write bytes by setting the _binary-mode_ `b`.  This is important because the response contents are bytes.  The default mode for `open` is `text-mode`. So make sure you use `b`.

We'll use the `with` statement _context-manager_ to open the file, write the contents of the response, and then close the file.  Once the data in `r.content` is written sucessfully to the file, or if there is an error, the file is closed by the _context-manager_.

We also need to prepend the `downloads` path to the filename.  We do this using `Path` from the `pathlib` package in the standard library.

In [ ]:
outfile = Path('downloads', icesat_id)
with open(outfile, 'wb') as f:
    f.write(r.content)

Check to make sure it is downloaded.

In [ ]:
ls -l ./downloads

`ATL03_20190622061415_12980304_004_01.h5` is an HDF5 file.  `xarray` can open this but you need to tell it which group to read the data from.  In this case we read the height data for ground-track 1 left-beam.

In [ ]:
outfile = Path('downloads', icesat_id)
ds = xr.open_dataset(icesat_id, group='/gt1l/heights')
ds

### Determine variables of interest: SST, ocean color, chemistry...

In [ ]:
response = requests.get(cmr_collection_url, 
                        params={
                            'concept_id': 'C1940475563-POCLOUD',
                            'cloud_hosted': 'True',
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()
variables = response['feed']['entry'][0]['associations']['variables']
output_format = "umm_json"
var_url = "https://cmr.earthdata.nasa.gov/search/variables"
for i in range(len(variables)):
    response = requests.get(f"{var_url}.{output_format}?concept-id={variables[i]}")
    response = response.json()
    # print(response['items'][0]['umm'])
    if 'Name' in response['items'][0]['umm']: pprint(response['items'][0]['umm']['Name'])

### Pull those variables into xarray "in place"

#### First, we need to determine the granules returned from our time and area of interest

In [ ]:
response = requests.get(granule_url, 
                        params={
                            'concept_id': 'C1940475563-POCLOUD',
                            'temporal': temporal,
                            'bounding_box': bounding_box,
                            'page_size': 200,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response.status_code)
print(response.headers['CMR-Hits'])

In [ ]:
modis_granules_meta = response.json()['feed']['entry']
for granule_meta in modis_granules_meta:
    print(granule_meta['boxes'])
    print(granule_meta['links'][0]['href'])

In [ ]:
https_link = modis_granules_meta[0]['links'][0]['href']
s3_link = https_link.replace('https://archive.podaac.earthdata.nasa.gov/','s3://')
s3_link

### Get S3 credentials

In [ ]:
s3_credentials = requests.get('https://archive.podaac.earthdata.nasa.gov/s3credentials').json()

In [ ]:
s3_fs = s3fs.S3FileSystem(
    key=s3_credentials["accessKeyId"],
    secret=s3_credentials["secretAccessKey"],
    token=s3_credentials["sessionToken"],
)

### Open a s3 file

In [ ]:
f = s3_fs.open(s3_link)
ds = xr.open_dataset(f, engine='h5netcdf')

### Use geolocation of ICESat-2 to define the single transect used to pull coincident ocean data out from array

### Create a plot of the single transect of gridded data 

(bonus: time series) - describe what this means to egress out of the cloud versus pulling the original data down (benefit to processing in the cloud)

## Download MODIS GHRSST data from Cloud

In [ ]:
from shapely.geometry import box

In [ ]:
map_proj = ccrs.PlateCarree()

In [ ]:
def bbox_geometry(boxs, t_crs=ccrs.NorthPolarStereo()):
    '''Generates a shapely.geometry.box object from boxes metadata'''
    lat_min, lon_min, lat_max, lon_max = [float(v) for v in bbox.split()]
    x_min, y_min = t_crs.transform_point(lon_min, lat_min, ccrs.PlateCarree())
    x_max, y_max = t_crs.transform_point(lon_max, lat_max, ccrs.PlateCarree())
    return box(x_min, y_min, x_max, y_max)

bbox_features = []
for granule in modis_granules_meta:
    for bbox in granule['boxes']:
        bbox_features.append(bbox_geometry(bbox))

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection=ccrs.NorthPolarStereo())
ax.set_extent([-180.,180.,60.,90.], ccrs.PlateCarree())
ax.coastlines()
ax.add_geometries([bbox_features[0]], crs=map_proj, alpha=0.3)

---

## Resources (optional)

---

## Conclusion